<a href="https://colab.research.google.com/github/aksanaboo/persona_predict/blob/master/MBTI_500_SVC_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.metrics import classification_report, f1_score
import pickle
import os.path
import plotly.offline as pyo
import plotly.graph_objs as go
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
mbti_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/MBTI 500.csv')


In [4]:
mbti_data = mbti_data.rename(columns={'type':'personality_type'})

In [5]:
recreate_model=False

In [6]:
filename = 'mbti500_SVCmodel.sav'

In [7]:
if not os.path.isfile(filename):
    recreate_model=True

In [8]:
X = mbti_data['posts']
y = mbti_data['personality_type']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [9]:
if recreate_model:


    # Split the data into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

    # Creating an instance of the vectorizer and training it
    vectorizer = TfidfVectorizer(max_features=5000)
    X_train_tfidf = vectorizer.fit_transform(X_train)
    X_val_tfidf = vectorizer.transform(X_val)

    # Training the classifier
    clf = LinearSVC(C=1.0, loss='hinge', random_state=42,max_iter=5000)
    clf.fit(X_train_tfidf, y_train)

    # Pipelining the vectorizer and the classifier
    text_clf = Pipeline([('tfidf', vectorizer), ('clf', clf)])
    text_clf.fit(X_train, y_train)

    # Save the model to disk
    with open(filename, 'wb') as model_file:
        pickle.dump(text_clf, model_file)

    # Evaluate the model on the validation set
    val_accuracy = text_clf.score(X_val, y_val)
    print("Validation Accuracy:", val_accuracy)

# Load the model from disk
with open(filename, 'rb') as model_file:
    text_clf = pickle.load(model_file)

# Assuming X_test and y_test are your testing data
test_accuracy = text_clf.score(X_test, y_test)
print("Test Accuracy:", test_accuracy)



Validation Accuracy: 0.8189225589225589
Test Accuracy: 0.8188617579585808


In [10]:
predictions = text_clf.predict(X_test)

In [11]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

        ENFJ       0.77      0.61      0.68       484
        ENFP       0.82      0.76      0.79      1834
        ENTJ       0.84      0.74      0.79       907
        ENTP       0.83      0.81      0.82      3494
        ESFJ       0.43      0.33      0.38        54
        ESFP       0.60      0.45      0.51       103
        ESTJ       0.81      0.82      0.81       157
        ESTP       0.89      0.89      0.89       595
        INFJ       0.81      0.82      0.82      4426
        INFP       0.79      0.81      0.80      3612
        INTJ       0.82      0.86      0.84      6690
        INTP       0.83      0.87      0.85      7604
        ISFJ       0.68      0.54      0.60       203
        ISFP       0.68      0.55      0.61       242
        ISTJ       0.75      0.60      0.66       384
        ISTP       0.85      0.78      0.81      1032

    accuracy                           0.82     31821
   macro avg       0.76   

In [12]:
print(f"Overall accuracy of the model: {round(metrics.accuracy_score(y_test, predictions),2)}")

Overall accuracy of the model: 0.82
